# <font color=#0404B4>**SEMINAR - PHÂN TÍCH DỮ LIỆU THÔNG MINH** </font>

### <font color=#0404B4> Bảng thành viên nhóm </font>
|<center><div style="width:150px">MSSV</div><center>|<center><div style="width:290px">Họ và tên</div><center>|
|---------- |:-------------:|
| <center>20120032<center>  | <center>Phan Trường An<center> |
| <center>20120037<center>  | <center>Trần Thị Minh Anh<center> |
| <center>20120041<center>  | <center>Trần Kim Bảo<center> |
| <center>20120053<center>  |   <center>Nguyễn Thành Đạt<center>   | 
| <center>20120071<center>  |<center>Nguyễn Thị Bích Hà<center> |
| <center>20120095<center>  | <center>Ngô Quang Hưng<center> |

# <font color=#0404B4>Import thư viện</font>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud
sns.set_theme()
pd.set_option("display.max_colwidth", 250)

In [ ]:
# !pip install locationtagger
# !pip install geotext
# !pip install geopy
# pip install matplotlib wordcloud

In [ ]:
from geotext import GeoText
from geopy.geocoders import Nominatim

<a name="2"></a>
# <font color=#0404B4> A. Tiền xử lý dữ liệu</font>


### 1. Đọc dữ liệu

In [ ]:
train_df=pd.read_csv("train.csv")
train_df.shape

In [ ]:
test_df=pd.read_csv("test.csv")
test_df.shape

In [ ]:
sample_submission_df=pd.read_csv("sample_submission.csv")
sample_submission_df.shape

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
sample_submission_df.head()

### 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?
- `train_df`: Thông tin của tweet trên tập huấn luyện gồm: id, từ khóa, địa điểm gửi, nội dung và nhãn của tweet.
- `test_df`: Thông tin của tweet trên tập test gồm: id, từ khóa, địa điểm gửi, nội dung.
- `sample_submission_df`: định dạng nộp bài.

### 3. Có dữ liệu bị trùng lặp ở các dataframe không

In [ ]:
def check_duplicated(dataframe):
    duplicated_rows = dataframe.duplicated()

    # In các hàng bị lặp lại
    if len(dataframe[duplicated_rows])>0:
        print('Có dòng dữ liệu bị lặp')
        display(dataframe[duplicated_rows])
    else: 
        print('Không có dòng dữ liệu nào bị lặp')


Dataframe `train_df`

In [ ]:
check_duplicated(train_df)

Dataframe `test_df`

In [ ]:
check_duplicated(test_df)

Dataframe `sample_submission_df`

In [ ]:
check_duplicated(sample_submission_df)

<h3> 4. Mỗi cột có ý nghĩa gì?</h3>

|STT|Tên cột dữ liệu | <center>Mô tả<center> | Ghi chú |
|--|:------:| ---| --- |
|1|id|<center>định danh của tweet<center>|
|2|keyword| <center>Từ khóa được lấy từ tweet<center>|Có thể thiếu|
|3|location|<center>Địa điểm đăng tweet<center>| Có thể thiếu|
|4|text| <center>Nội dung của tweet<center>||
|5|target|<center>Nhãn được gán cho tweet<center>|1: Thảm họa thực sự <br> 0: Không xảy ra thảm họa| 



### 5. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp?



Dataframe `train_df`

In [ ]:
train_df.columns

In [ ]:
train_df.info()

Dataframe `test_df`

In [ ]:
test_df.columns

In [ ]:
test_df.info()

- Nhận xét:
    - Dựa vào ý nghĩa của id và kích thước của `train_df` và `test_df`, nhận thấy rằng chúng ta có thể merge dữ liệu ở 2 dataframe để tạo thành 1 dataframe mới: `tweet_df`. Để tiện cho phần khám phá dữ liệu.

In [ ]:
tweet_df=pd.concat([train_df,test_df])

In [ ]:
tweet_df=tweet_df.sort_values(by="id").reset_index(drop=True)

In [ ]:
check_duplicated(tweet_df)

In [ ]:
tweet_df.columns

In [ ]:
tweet_df.info()

### 6. Với mỗi cột, các giá trị được phân bố như thế nào?

#### Cột có kiểu dữ liệu dạng số, và phân bố:


In [ ]:
numerical_cols = tweet_df[tweet_df.select_dtypes(exclude=['object']).columns]
numerical_cols.head(5)

- Với các cột dữ liệu dạng số ta sẽ tính % dữ liệu thiếu, min, max. Kết quả được lưu vào dataframe `nume_col_profiles_df`, với:
    - `missing_ratio`: % dữ liệu thiếu.
    - `min`: giá trị nhỏ nhất.
    - `max`: giá trị lớn nhất.

In [ ]:
numerical_cols = tweet_df[tweet_df.select_dtypes(exclude=['object']).columns]

def missing_ratio(c):
    return  c.isna().sum()/len(c) * 100

def min(c):
    return c.min()

def max(c):
    return c.max()

nume_col_profiles_df = numerical_cols.agg([missing_ratio, min, max])
nume_col_profiles_df

- Nhận xét:
    - `id`: Không có dữ liệu thiếu, ID tweet nhỏ nhất là 0, lớn nhất là 10875.
    - `target`: Có 30% dữ liệu thiếu tương ứng với các tweet chưa được gán nhãn ở `test_df`.

#### Cột có kiểu dữ liệu phân loại, và phân bố:


In [ ]:
object_cols = tweet_df[tweet_df.select_dtypes(include=['object']).columns]
object_cols.head(5)

- Với các cột categorical, ta tính tỉ lệ % dữ liệu thiếu, số lượng giá trị khác nhau, phân bố của các giá trị khác nhau. Kết quả được lưu vào dataframe `cate_col_profiles_df`, với:
    - `missing_ratio`: % dữ liệu thiếu
    - `num_diff_vals`: số lượng giá trị khác nhau <em>(không xét giá trị thiếu)</em>
    - `ratio_diff_vals`: phân bố của các giá trị khác nhau <em>(không xét giá trị thiếu)</em>.

In [ ]:
def num_diff_vals(col):
    return len(col.value_counts())

def ratio_diff_vals(col):
    return np.round(col.value_counts(normalize=True),2).to_dict()

cate_col_profiles_df=object_cols.agg([missing_ratio,num_diff_vals,ratio_diff_vals])
cate_col_profiles_df

- Nhận xét:
    - `location`: xuất hiện nhiều tweet không có địa chỉ đăng(33,45%), số lượng địa điểm lớn: 4521 địa điểm.
    - `keyword`: vẫn xuất hiện các tweet không trích dẫn được từ khóa, số lượng không quá lớn: 221 cụm từ khóa.
    - `text`: Không bị thiếu dữ liệu, nhưng số lượng nội dung chỉ là 10678 < 10875 (số lượng bài tweet). Có thể là do 1 số sự kiện/vấn đề nào đó.

###  7. Có cột nào cần phải tiền xử lý hay không?


- Nhận xét:
    - Qua kiểm tra dữ liệu, ta có thể thấy ở thuộc tính `location` có rất nhiều giá trị 4521 xấp xỉ bằng 1/2 tổng 2 tập `train_df` và `test_df` cộng lại. Thực hiện kiểm tra và xử lý về dạng chuẩn nhất có thể.

#### 1. Xóa các địa điểm bị lỗi ký tự, xuất hiện ký tự đặc biệt nhiều:


Thực hiện xóa các hàm có ký tự đặc biệt nhiều hơn 1/4 so với độ dài của chuỗi. Và nếu trong cell đang xét có từ nào trên 2 ký tự thì thực hiện trả về *(Chỉ lấy 1 từ cuối)*

In [ ]:
def del_special_string(c):
  if not(c is np.nan):
    special_char=re.sub('[\w^\s\.]+' ,'', c)
    scale=len(special_char)/len(c)
    if scale > 1/4:
      words=re.findall('[a-zA-Z]{2,}',c)
      if len(words)>0:
        return words[-1]
      return np.nan
    else:
      return c
  return np.nan

Thực hiện chạy hàm `del_special_string` cho tất cả các dữ liệu trong cột `location`

In [ ]:
tweet_df.location.apply(del_special_string).str.strip().replace('',np.nan).unique().__len__()

In [ ]:
tweet_df.location=tweet_df.location.apply(del_special_string).str.strip().replace('',np.nan)

Đếm số lần địa điểm được sử dụng

In [ ]:
location_count_df=tweet_df.location.value_counts().to_frame().reset_index().rename(columns={"location":"counts","index":"location"})
location_count_df

In [ ]:
location_array=location_count_df.location

#### 2.  Lọc các dữ liệu có tên thành phố hoặc tên quốc gia

##### Trích những tên thành phố và quốc gia *(đúng chuẩn)*

Tạo dataframe chứa thông tin: thành phố và quốc gia dựa trên các địa chỉ *(unique)*

In [ ]:
location_dict={"city":[],"country":[]}
for location_str in location_array:

  if isinstance(location_str, str):
    places = GeoText(location_str)

    if len(places.cities)>0:
      if len(places.cities)>1:
        result=', '.join(places.cities)
        location_dict["city"].append(result)
      else :
        location_dict["city"].append(places.cities[0])
    else :
        location_dict["city"].append(np.nan)

    if len(places.countries)>0:
      location_dict["country"].append(places.countries[0])
    else :
      location_dict["country"].append(np.nan)
  else:
    location_dict["city"].append(np.nan)
    location_dict["country"].append(np.nan)
  

In [ ]:
location_df=pd.DataFrame(location_dict)
location_df

##### Điền tên quốc gia cho các tweet chỉ có tên thành phố

In [ ]:
def get_country(city):
  if not (city is np.nan):
    geolocator = Nominatim(user_agent="my_app")
    location = geolocator.geocode(city)
    if location:
      return location.raw['display_name'].split(",")[-1].strip()
    return np.nan
  return np.nan

In [ ]:
location_fill_country=location_df[~(location_df.city.isna()) & (location_df.country.isna())]

In [ ]:
# Đã chạy và lưu kết quả vào file fill_country.csv
countries = location_fill_country.city.iloc[:10].apply(get_country)

location_df.loc[countries.index, 'country'] = countries.values
location_df

Xuất file `fill_country.csv` sau khi đã điền: để thuận tiện cho các lần chạy sau

In [ ]:
location_df["location"]=pd.Series(location_array)

In [ ]:
# location_df.to_csv("fill_country.csv",index=False,header=True)

Đọc lại dữ liệu từ `fill_country.csv`

In [ ]:
location_df=pd.read_csv("fill_country.csv")
location_df

#### 3. Chuẩn hóa các thông tin bị trống cả tên thành phố và quốc gia

Lọc các tweet bị trống thông tin ở 2 cột thành phố và quốc gia

In [ ]:
location_nan_all=location_df[(location_df.city.isna()) & (location_df.country.isna())]
location_nan_all

Hàm trả về tên quốc gia cho location chưa chuẩn hóa *(nếu có đề cập)*

In [ ]:
def get_country_nan_all(c):
  geolocator = Nominatim(user_agent='myencoder', timeout=10)
  first = geolocator.geocode(c)
  if first:
    place=GeoText(first.address)
    if len(place.countries)<1:
      return first.raw['display_name'].split(",")[-1].strip()
    else :
      return place.countries[0]
  else :
    return np.nan

In [ ]:
location_count_df[location_count_df.counts>1].counts.sum()

- Nhận xét:
    - Thực hiện chuẩn hóa cho các địa điểm xuất hiện nhiều hơn 1 lần.
    - Theo tìm hiểu để có thể lấy được tên các quốc gia chưa được chuẩn hóa trong ngôn ngữ tự nhiên. Nhóm quyết định xử dụng thư viện `geopy`: truy cập lấy từ API từ [OpenStreetMap Nominatim](https://nominatim.openstreermap.org). Cho phép lấy được cả các thông tin chưa được chuẩn hóa dựa trên gợi ý từ API. Tuy nhiên với điều khoản chỉ có thểm lấy 1 địa chỉ/s và bị giới hạn số lần yêu cầu. Nên khó khăn trong việc chuẩn hóa toàn bộ.


In [ ]:
len_NanCountrySuggest=location_count_df[location_count_df.counts>1].shape[0]

In [ ]:
location_nan_all[location_nan_all.index<len_NanCountrySuggest]

In [ ]:
start_idx=0
end_idx=100
steps=100
# n=location_nan_all[location_nan_all.index<len_NanCountrySuggest].shape[0]
n=0
while(True):
    if end_idx>n:
        countries = location_nan_all.location.iloc[start_idx:n].apply(get_country_nan_all)
        location_df.loc[countries.index, 'country'] = countries.values
        break
    countries = location_nan_all.location.iloc[start_idx:end_idx].apply(get_country_nan_all)
    location_df.loc[countries.index, 'country'] = countries.values
    print(end_idx)
    start_idx+=steps
    end_idx+=steps

In [ ]:
# location_df.to_csv("fill_location.csv",index=False,header=True)

In [ ]:
location_nomalize=pd.read_csv("fill_location.csv")

In [ ]:
location_nomalize

<a name="2"></a>
# <font color=#0404B4> B. Khám phá dữ liệu</font>


#### Note: T chỉ mới chuẩn hóa ở các địa chỉ unique nên trong toàn bộ dữ liệu muốn thay đổi là phải replace dựa trên location của dataframe location_nomalize. 

### Case 1: tìm ra các target bằng 1 hay có những keyword nào?

**Bước 1:** Tiền xử lý dữ liệu
Ta tiến hành lọc ra các sample có keyword không phải NaN và có target bằng 1 (disater).

In [ ]:
case1Train = train_df
# case1Train.head()
trainTarget1= case1Train[case1Train['keyword'].notnull()]
trainTarget1= trainTarget1[trainTarget1['target']==1]
trainTarget1.head()

# trainTarget1.dtypes
# trainTarget1.tail()


**Bước 2:** Trực quan hóa bằng word cloud

In [ ]:
# Assuming your data is in a pandas DataFrame called 'trainTarget1'
keywords = trainTarget1['keyword'].dropna().tolist()

# Concatenate the keywords into a single string
text = ' '.join(keywords)

# Create a word cloud object
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Plot the word cloud
plt.figure(figsize=(16, 9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

**Bước 3:** Để có thể quan sát rõ hơn về số lần keyword xuất hiện, ta thêm 1 bước trực quan hóa dữ liệu bằng biểu đồ cột. Trong đó trục x thể hiện `keyword`, trục y thể hiện `số lần` keyword đó xuất hiện

In [ ]:
keywords_counts = trainTarget1['keyword'].value_counts().sort_values(ascending=False).head(10)

# Plotting the bar chart
plt.figure(figsize=(16, 6))
keywords_counts.plot(kind='bar')
plt.xlabel('Keyword',fontsize=5)
plt.ylabel('Frequency')
plt.title('Keyword Frequency')
plt.xticks(rotation=45,fontsize=8)
plt.show()

***Nhận xét***:
- Quan sát biểu đồ, số lần xuất hiện của mỗi keyword ở top 10 khá đồng đều, không có sự chênh lệch nhiều.

### Case 2: Liệu có những mối quan hệ nào giữa `location` và cột `target`?

**Import thư viện**

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot

**Bước 1:** Tiền xử lý dữ liệu

Vì cột `location` có quá nhiều dữ liệu thiếu nên ta xử lý bằng cách phân ra, địa điểm đã biết và địa điểm chưa được xác định.

In [ ]:
train_ex=train_df.copy()
train_ex['keyword']=train_ex['keyword'].fillna('Unknown')
train_ex['location']=train_ex['location'].fillna('Unknown')
train_ex['location']=train_ex['location'].apply(lambda x: 'Known' if x != 'Unknown' else x)

location_count=train_ex['location'].value_counts()
print(location_count)
location=location_count.keys()
lcount=location_count.values

**Bước 2:** Trực quan bằng `donut chart`

`Donut chart` rất phù hợp để quan sát tỉ lệ tweet có địa điểm đã biết và chưa biết.

In [ ]:
pie_chart1=go.Pie(labels=location,values=lcount,hole=0.6)

my_layout=go.Layout(title='Biểu đồ biểu diễn tỉ lệ mức độ nghiêm trọng theo từng kiểu địa điểm',
                    height=500,width=800)

iplot({'data':pie_chart1, 'layout':my_layout})


***Nhận xét***:
- Quan sát biểu đồ, ta thấy lượng địa điểm chưa biết chiếm 1/3 so với tổng thể.
> Dữ liệu thiếu khá nhiều, cần trực quan kỹ hơn để đưa ra nhận xét. Ta xác định số lượng địa điểm biết và chưa biết dựa trên 2 nhóm tweet 1 và 0.


**Bước 3:** Tính các tham số cần thiết cho việc trực quan.

In [ ]:
unknown=train_ex.loc[train_ex['location']=='Unknown']['target'].value_counts()
known=train_ex.loc[train_ex['location']=='Known']['target'].value_counts()
utarget=unknown.keys()
ucount=unknown.values
ktarget=known.keys()
kcount=known.values

target_count=train_ex['target'].value_counts()
target=target_count.keys()
tcount=target_count.values
total=sum(tcount)

**Bước 4**: Trực quan bằng `Nest chart`.

`Nest chart` được dùng khi cần các mối quan hệ cha và con trong biểu đồ.

In [ ]:
import plotly.graph_objs as go

data = [
go.Pie(values=[tcount[1],tcount[0]],
    labels=[1,0],
    domain={'x':[0.2,0.8], 'y':[0.1,0.9]},
    hole=0.5,
    direction='clockwise',
    sort=False,
    marker={'colors':['#CB4335','#2E86C1']},
    ),
   
    go.Pie(values=[kcount[1],ucount[1],kcount[0],ucount[0]],
    labels=['Known-1','Unknown-1','Known-0','Unknown-0'],
    domain={'x':[0.1,0.9], 'y':[0,1]},
    hole=0.75,
    direction='clockwise',
    sort=False,
    marker={'colors':['#EC7063','#F1948A','#5DADE2','#85C1E9']},
    showlegend=False)]
my_layout = go.Layout(title='Biểu đồ biểu diễn tỉ lệ địa điểm có thông tin và chưa có thông tin dựa trên tweet 0 và 1',
                      height=500, width=800,
                      annotations=[{
                          'font': {
                              'size': 20
                          },
                          'showarrow': False,
                          'text':f'{total}',
                          'hovertext': f'Total: {total}',
                          'x': 0.5,
                          'y': 0.5,
                        
                      }])

fig = go.Figure(data=data, layout=my_layout)

iplot(fig)

***Nhận xét***:
- Với tweet 1 và tweet 0, lượng địa điểm đã biết có xu hướng cao hơn khoảng gấp đôi so với địa điểm chưa biết. Điều này cũng khá tương đồng với tỉ lệ khi trực quan tổng thể.
> Có thể kết luận rằng, số lượng địa điểm đã biết và chưa biết hầu như không có liên hệ với lượng tweet o và 1. 
>> Thử liên hệ cột `location` với cột `target` theo cách khác với case 3.

### Case 3: Lúc tìm được `location` hay xảy ra thảm hoạ, biến cố, xem có các `keyword` là gì?

**Bước 1:** Tiền xử lý dữ liệu
Ta lọc ra các sample có `keyword` khác `NaN`, `target` bằng `1` và `location` khác `NaN`

In [ ]:
case3Train = train_df
# case3Train.head()
trainTarget3= case3Train[case3Train['keyword'].notnull()]
trainTarget3= trainTarget3[trainTarget3['target']==1]
trainTarget3= trainTarget3[trainTarget3['location'].notnull()]
trainTarget3.head()

# trainTarget1.dtypes
# trainTarget1.tail()


**Bước 2:** Ta tiến hành trực quan bằng biểu đồ cột, trong đó trục x thể hiện các `location`, trục y thể hiện `số lần` mà location đó xảy ra disater. Sắp xếp giảm dần ta được biểu đồ như bên dưới

In [ ]:
keywords_counts = trainTarget3['location'].value_counts().sort_values(ascending=False).head(10)

# Plotting the bar chart
plt.figure(figsize=(10, 6))
keywords_counts.plot(kind='bar')
plt.xlabel('Keyword',fontsize=5)
plt.ylabel('Frequency')
plt.title('Keyword Frequency')
plt.xticks(rotation=45,fontsize=8)
plt.show()

***Nhận xét***:
- Ta dễ dàng thấy được, USA là nơi xảy ra thảm họa nhiều nhất, với số lượng vượt trội so với các quốc gia khác trong top 10.
> Ta tiến hành phân tích các keyword có liên quan đến USA.

**Bước 4**: Từ dữ liệu đã lọc ở bước trên, ta thêm 1 bước lấy ra các sample có `location` là `USA`. Sau đó trực quan bằng biểu đồ cột, trục x thể hiện `keyword`, trục y thể hiện `số lần` keyword đó xuất hiện

In [ ]:
trainTarget3= trainTarget3[trainTarget3['location']=="USA"]

keywords_counts = trainTarget3['keyword'].value_counts().sort_values(ascending=False).head(10)

# Plotting the bar chart
plt.figure(figsize=(10, 6))
keywords_counts.plot(kind='bar')
plt.xlabel('Keyword',fontsize=5)
plt.ylabel('Frequency')
plt.title('Keyword Frequency')
plt.xticks(rotation=45,fontsize=8)
plt.show()


**Nhận xét:**
- Từ biểu đồ trên, ta dễ dàng nhận thấy `sandStorm` chiếm ưu thế hơn hẳn trong top 10, với số lượng vượt trội.

> Từ phân tích trên có thể rút ra được, nếu là USA thì khả năng xảy ra thảm họa "sandstorm" là khá cao.

### Case 4: Thống kê cột `text` theo số lượng chữ trên dòng, theo câu để kiểm tra có điểm bất thường nào không?



**Import thư viện**

In [ ]:
import nltk


**Bước 1**: Tạo cột đếm số lượng chữ của cột `text`.

In [ ]:
def word_count(sample):
    sample_list=sample.split()
    return len(sample_list)

In [ ]:
train_df['word_count']=train_df['text'].apply(word_count)

**Bước 2:** Tạo cột số lượng chữ và số lượng chữ trung bình của cột `text`.

In [ ]:
train_df['length']=train_df['text'].str.len()
train_df['mean_word_length']=train_df['text'].map(lambda rev: np.mean([len(word)for word in rev.split()]))

**Bước 3**: Tính số lượng chữ trung bình trên câu.

In [ ]:
train_df['mean_sentence_length']=train_df['text'].map(lambda rev: np.mean([len(sentence)for sentence in nltk.tokenize.sent_tokenize(rev)]))

**Bước 4**: Trực quan bằng `line chart` và `boxplot`.

- `Boxplot` phù hợp để kiểm tra độ phân bố dữ liệu của các cột số lượng từ, độ dài, số lượng từ trung bình và số lượng từ trung bình trên câu.
- `Line chart` phù hợp để kiểm tra biến động, giúp thể hiện rõ được biến động của lượng tweet 0 và 1 dựa trên số lượng từ, độ dài, số lượng từ trung bình và số lượng từ trung bình trên câu.

In [ ]:
def visualize(col):
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))  # Create a figure with a larger size
    
    sns.boxplot(y=train_df[col], hue=train_df['target'], ax=axes[0])  # Plot the boxplot on the left subplot
    axes[0].set_ylabel(col, labelpad=12.5)
    
    sns.kdeplot(train_df[col], hue=train_df['target'], ax=axes[1])  # Plot the kdeplot on the right subplot
    axes[1].legend(train_df['target'].unique())
    axes[1].set_xlabel('')
    axes[1].set_ylabel('')
    plt.show()

In [ ]:
features = train_df.columns.tolist()[5:]
for feature in features:
    visualize(feature)

***Nhận xét***:
- 
-
- 
-  

### Case 5: Tương quan 2 cột `keyword` và `text` để tìm ra mối liên hệ với tweet 1.

**Download thư viện**

In [ ]:
# !pip install WordCloud

**Import thư viện**

In [ ]:
from wordcloud import WordCloud,STOPWORDS 
from PIL import Image

**Bước 1**: Trực quan bằng `wordcloud` những dòng có tweet 1.

In [ ]:
fpath=("GothamMedium.ttf")

mask = np.array(Image.open("realcloud.png"))
mask=~mask
train_text=train_df.loc[train_df['target']==1].text
train_text=list(train_text.values)
text = " ".join(cat for cat in train_text)
word_cloud = WordCloud(stopwords = STOPWORDS,
                        mask = mask, 
                        background_color = 'white',
                        contour_width=15,
                        contour_color='slateblue',
                        font_path=fpath,
                        max_font_size=600,
                        max_words=200,
                        width = mask.shape[0],
                        height = mask.shape[1],
                        colormap='magma'
                        )
word_cloud.generate(text)

In [ ]:
# Display the generated Word Cloud
plt.figure(figsize=(16,9),facecolor='lightblue')
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**Bước 2**: Tìm những `keyword` thường xuất hiện cột `text`trên tweet 1.

In [ ]:
word_list = word_cloud.process_text(text)

word_df={}
word_df['word']=word_list.keys()
word_df['count']=word_list.values()
word_df=pd.DataFrame(word_df)
word_df=word_df.sort_values('count',ascending=False).head(5000)
keywords_counts = trainTarget1['keyword'].value_counts().sort_values(ascending=False).head(30)
k=[]
for i in keywords_counts.keys():
    word_check=word_df.loc[word_df['word'].str.contains(i)]
    if word_check.shape[0]!=0:
        k.append(i)
k=np.array(k)
k=keywords_counts.keys()
k

**Bước 3**: Đếm số lượng tweet 0 và 1 dựa trên các keyword.

In [ ]:
t0=0
t1=1
for i in k:
    target_counts=train_ex.loc[train_ex['text'].str.contains(i)].target.value_counts()

    for j in target_counts.keys():
        if j == 0:t0+=target_counts[j]
        else:t1+=target_counts[j]
print(t0)
print(t1)

**Bước 4:** Trực quan bằng `Donut chart`

`Donut chart` phù hợp để thể hiện tỉ lệ số lượng tweet 0 và 1 dựa trên các `keyword` đã tìm được.

In [ ]:
target=[1,0]
count=[t1,t0]

pie_chart=go.Pie(labels=target,values=count,hole=0.5,pull=[0, .2])

my_layout=go.Layout(title='Biểu đồ biểu diễn tỉ lệ mức độ nghiêm trọng dựa trên các từ đã tương quan',
                    height=500,width=800)
iplot({'data':pie_chart,'layout':my_layout})


***Nhận xét:***
- Tỉ lệ tweet 1 cao hơn tweet 0 cao hơn rất nhiều dựa trên các `keyword` kiếm được.
> Nhưng liệu có bị `overfit` không khi thực hiện như vậy. Tiếp theo, phần `model` sẽ tiến hành tìm hiểu xem.

### Model